# Simple Q & A Bot
## Overview
This is a Simple Q & A Bot that uses Gemini and LangChain to answer questions. The agent is designed to understand user questions and provide accurate answers based on the provided context.

## Motivation
To learn fundamentals of building agents. Creating a Q & A bot is a great way to get started with building agents.
- Demonstrate the use of LangChain and Gemini.
- Show how to build a simple Q & A bot.
- provide a foundation for more complex agents.


## Key Components
1. Language Model: Gemini
2. LangChain: A framework for building agents.
3. Prompt template: for defining the structure of the question and context to the language model.
4. LLMChain: for chaining the language model and prompt template.

## Method Details
1. Setup and Initialization
2. Define the Prompt Template
3. Creating the LLM Chain
4. Implementing Q & A function
5. User Interaction


## Conclusion
This Simple Q & A bot is a great way to get started with building agents. It is a simple and easy to understand example of how to build a Q & A bot using LangChain and Gemini. I can add few more things like:
- Adding memory to maintain context across multiple interactions.
- Adding a knowledge base to provide context to the language model.
- Adding a chat interface to interact with the bot.




##

In [47]:
import os
from dotenv import load_dotenv
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate    
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory,  ChatMessageHistory
load_dotenv()

True

In [2]:
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # Changed model here!
    temperature=0,
    google_api_key=os.getenv('GOOGLE_API_KEY')
)

In [8]:
store = {}

def getChatHistory(session_id:str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [55]:
template = """"
You are a helpful assistant. Your task is to answer the user's questions to the best of your knowledge and chat history.
Question: {question}
Please provide a clear and concise answer:
Answer: """

prompt = PromptTemplate(template=template, question_variables=["question"], input_variables=["history"]) 

In [56]:
qa_chain = LLMChain(llm=llm, prompt=prompt)

In [57]:
chain_with_history = RunnableWithMessageHistory(
    qa_chain,
    getChatHistory,
    input_messages_key="question",
    history_messages_key="history"
)

In [58]:
def get_answer(question):
    """Get an answer to the given question using the LLM chain."""
    question_variables = {"question": question}
    response = qa_chain.run(question_variables)
    return response

In [60]:
question = "what is the capital of India?"
answer = get_answer(question)
print("Question:", question)
print("Answer:", answer)

Question: what is the capital of India?
Answer: New Delhi


In [61]:

user_question = input("Enter your question:")
user_answer = get_answer(user_question)
print(f"Answer: {user_answer}")

Answer: Answer:  I need more information. "Ho" is incomplete.  Please clarify your question.


In [62]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"question": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print(response1)


response2 = chain_with_history.invoke(
    {"question": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print(response2)


{'question': 'Hello! How are you?', 'history': [HumanMessage(content='Hello! How are you?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am doing well, thank you!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What was my previous message?', additional_kwargs={}, response_metadata={}), AIMessage(content='Answer: I have no memory of past conversations.  Each interaction with me starts fresh.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello! How are you?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am doing well, thank you!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What was my previous message?', additional_kwargs={}, response_metadata={}), AIMessage(content='Answer: I have no memory of past conversations.  Each interaction with me starts fresh.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello! How are you?', additional_kwargs={}, response_metad

In [63]:
# Example usage:
session_id = "user_321" # A unique identifier for this conversation session

print("First interaction:")
response1 = chain_with_history.invoke(
    {"question": "Hello! How are you today?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1)

print("\nSecond interaction (referencing previous message):")
response2 = chain_with_history.invoke(
    {"question": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2)

print("\nThird interaction with a different session (should not remember previous):")
session_id_2 = "user_456"
response3 = chain_with_history.invoke(
    {"question": "What's the capital of France?"},
    config={"configurable": {"session_id": session_id_2}}
)
print("AI:", response3)

print("\nFourth interaction with original session (should remember its own history):")
response4 = chain_with_history.invoke(
    {"question": "And what about my first message to you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response4)

First interaction:
AI: {'question': 'Hello! How are you today?', 'history': [HumanMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am doing well, thank you!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What was my previous message?', additional_kwargs={}, response_metadata={}), AIMessage(content='Answer: I have no memory of past conversations.  Each interaction with me starts fresh.', additional_kwargs={}, response_metadata={}), HumanMessage(content='And what about my first message to you?', additional_kwargs={}, response_metadata={}), AIMessage(content='Answer: I have no memory of past conversations.  Each interaction with me starts fresh.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello! How are you today?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am doing well, thank you for asking! How are you today?', additional_kwargs={}, response_metadata={}),